In [1]:
import cv2
import os
import json
import numpy as np
import pandas as pd
from PIL import Image, ImageChops, ImageEnhance, ImageDraw
from mtcnn import MTCNN

In [2]:
train_sample_metadata = pd.read_json('../input/dfdc_train_part_0/metadata.json').T
train_sample_metadata

,label,split,original
owxbbpjpch.mp4,FAKE,train,wynotylpnm.mp4
vpmyeepbep.mp4,REAL,train,NaN
fzvpbrzssi.mp4,REAL,train,NaN
htorvhbcae.mp4,FAKE,train,wclvkepakb.mp4
fckxaqjbxk.mp4,FAKE,train,vpmyeepbep.mp4
...,...,...,...
hectzcuzdv.mp4,FAKE,train,vtunvalyji.mp4
yxkbjxmtzr.mp4,FAKE,train,sttnfyptum.mp4
wkdnagybtb.mp4,FAKE,train,jytrvwlewz.mp4
fonrexmbzz.mp4,FAKE,train,fufcmupzen.mp4


In [7]:
# train 영상 path
train_sample = '../input/dfdc_train_part_0/'
train_video_files = [train_sample + x for x in sorted(os.listdir(train_sample)) if x.endswith('.mp4')]

In [9]:
len(train_video_files)

1334

In [10]:
# original이 있는 FAKE
train_sample_metadata['FAKE has original'] = [(train_sample_metadata.iloc[:,2][i] in list(train_sample_metadata.index)) for i in train_sample_metadata.T]

In [11]:
# original이 있는 FAKE 수
len(train_sample_metadata[train_sample_metadata['FAKE has original']]['original'])

1248

In [12]:
# FAKE가 있는 origin 수
len(train_sample_metadata[train_sample_metadata['FAKE has original']]['original'].unique())

86

In [13]:
train_sample_metadata['original'].value_counts()

fdpisghkmd.mp4    36
fnslimfagb.mp4    35
wynotylpnm.mp4    34
fsaronfupy.mp4    33
uprwuohbwx.mp4    32
                  ..
ngvcqxjhyb.mp4     2
gcdtglsoqj.mp4     1
rmuxlgsedw.mp4     1
fgobmbcami.mp4     1
khtwrijuqn.mp4     1
Name: original, Length: 86, dtype: int64

In [24]:
# FAKE가 있는 origin 영상 리스트
original_vidio_has_fake = train_sample_metadata[train_sample_metadata['FAKE has original']]['original'].unique()
original_vidio_has_fake

array(['wynotylpnm.mp4', 'wclvkepakb.mp4', 'vpmyeepbep.mp4',
       'jawgcggquk.mp4', 'qarqtkvgby.mp4', 'sttnfyptum.mp4',
       'kqlvggiqee.mp4', 'lzbmwwejxb.mp4', 'jytrvwlewz.mp4',
       'wcqvzujamg.mp4', 'exxqlfpnbz.mp4', 'nweufafotd.mp4',
       'jvtjxreizj.mp4', 'gvasarkpfh.mp4', 'prmwoaeeng.mp4',
       'gnmmhlbzge.mp4', 'fopjiyxiqd.mp4', 'pqdeutauqc.mp4',
       'wfzjxzhdkj.mp4', 'dxfdovivlw.mp4', 'ddtbarpcgo.mp4',
       'dhjnjkzuhq.mp4', 'xmkwsnuzyq.mp4', 'vokrpfjpeb.mp4',
       'doniqevxeg.mp4', 'aayrffkzxn.mp4', 'upmgtackuf.mp4',
       'nrnklcxdzq.mp4', 'jzupayeuln.mp4', 'fnslimfagb.mp4',
       'prdrkaxeob.mp4', 'qyqufaskjs.mp4', 'psjfwjzrrh.mp4',
       'xjzkfqddyk.mp4', 'eiwtggvtfp.mp4', 'syxobtuucp.mp4',
       'vtunvalyji.mp4', 'fsaronfupy.mp4', 'hivnldfvyl.mp4',
       'uqtqhiqymz.mp4', 'rktrpsdlci.mp4', 'xkfliqnmwt.mp4',
       'ybnucgidtu.mp4', 'xchzardbfa.mp4', 'ybbrkacebd.mp4',
       'ldtgofdaqg.mp4', 'onbgbghesu.mp4', 'chqqxfuuzi.mp4',
       'heiyoojifp.mp4',

In [25]:
# path
original_vidio_has_fake_path = train_sample + original_vidio_has_fake

In [26]:
# 0번 origin 영상으로 만든 fake 영상들 => 6개
train_sample_metadata[train_sample_metadata['original']==original_vidio_has_fake[0]].index

Index(['owxbbpjpch.mp4', 'bpguesjrfa.mp4', 'skewcclbhg.mp4', 'ybdtkypwez.mp4',
       'qwvxbksoeo.mp4', 'epwesrdmpt.mp4', 'fybkxkgstv.mp4', 'mpuugiqewv.mp4',
       'toinozytsp.mp4', 'kihrpdtbwy.mp4', 'zvuidmemcp.mp4', 'qzchhaoptt.mp4',
       'mmuolksboc.mp4', 'phjlaexgcl.mp4', 'pllpxvzpok.mp4', 'fxredodlkl.mp4',
       'grqyyhfexr.mp4', 'jxvoarslil.mp4', 'mzwfaambxx.mp4', 'qypedtpatd.mp4',
       'vxuhkqqaqv.mp4', 'vbcjbecyos.mp4', 'noavesktql.mp4', 'achdeirhym.mp4',
       'fhzdtfiqrh.mp4', 'bkcztuqxbp.mp4', 'tptalpxwlv.mp4', 'ycpuhmcdom.mp4',
       'alpkghdgrv.mp4', 'wdyzqwntoe.mp4', 'sfbnrbvbpn.mp4', 'cbybbxuusl.mp4',
       'xvzjzkrmlm.mp4', 'dsgswagtvp.mp4'],
      dtype='object')

In [27]:
# 경로 만들어주기 (kaggle kernel에서 필요)
try:
    os.makedirs('../working/dataset/FAKE3')
except:
    pass
try:
    os.makedirs('../working/dataset/REAL3')
except:
    pass

In [28]:
import matplotlib.pyplot as plt
%matplotlib inline 

In [30]:
# origin 영상들로 학습시작
#detector = dlib.get_frontal_face_detector()
train_video_files = sorted(original_vidio_has_fake)

#train_video_files = ['bdnaqemxmr.mp4']

vid_num = 0
all_vid = len(train_video_files)
# origin 영상 얼굴찾기
for vid in train_video_files:
    count = 0
    path = train_sample + vid
    file_name = vid.split('.')[0]
    face_coord = {}
    cap = cv2.VideoCapture(path)
    frame = 0
    mtcnn = MTCNN() # device='cuda:0',
    
    before_face_img_coord = []
    
    while(cap.isOpened()):
        ret = cap.grab()
        if ret == False:
            break
        #if frame % 30 == 0:
        ret, image = cap.retrieve()
        #img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img = image
        face_coord[frame]=[]
        # 처음 얼굴이 나오면
        if not before_face_img_coord:
            faces = mtcnn.detect_faces(img)
            # 얼굴 한개 이상?
            for face_idx, face in enumerate(faces):
                if 'confidence' in face and face['confidence'] > 0.9:
                    x1,y1,w,h = face['box']
                    x2 = min(x1+w, img.shape[1])
                    y2 = min(y1+h, img.shape[0])
                    x1 = max(x1, 0)
                    y1 = max(y1, 0)
                    crop_img = img[y1:y2, x1:x2]
                    face_coord[frame].append([y1,y2,x1,x2])

                    #plt.imshow(crop_img)
                    #plt.show()
                    cv2.imwrite(f'../working/dataset/REAL3/REAL_{file_name}_{frame}_{face_idx}.jpg', cv2.resize(crop_img, (299, 299)))
                    before_face_img_coord.append([y1,y2,x1,x2])
                    
                    break
                    
        # 이전에 검출된 face가 있으면
        else:
            for face_idx, coord in enumerate(before_face_img_coord):
                im = img[max(coord[0]-100,0) : min(coord[1]+100,img.shape[0]),
                         max(coord[2]-100,0) : min(coord[3]+100,img.shape[1])]
                face = mtcnn.detect_faces(im)
                if face:
                    # 1개만 저장해
                    x11,y11,w,h = face[0]['box']
                    x1 = max(coord[2]-100 + x11, 0)
                    x2 = min(x1 + w, img.shape[1])
                    y1 = max(coord[0]-100 + y11, 0)
                    y2 = min(y1 + h, img.shape[0])
                    crop_img2 = img[y1:y2, x1:x2]
                    
                    
                    face_coord[frame].append([y1,y2,x1,x2])
                    before_face_img_coord[face_idx]=[y1,y2,x1,x2]
                    
                    #plt.imshow(crop_img2)
                    #plt.show()
                    cv2.imwrite(f'../working/dataset/REAL3/REAL_{file_name}_{frame}_{face_idx}.jpg', cv2.resize(crop_img2, (299, 299)))
                    crop_img = crop_img2
                else:
                    face_coord[frame].append([y1,y2,x1,x2])
                    cv2.imwrite(f'../working/dataset/REAL3/REAL_{file_name}_{frame}_{face_idx}.jpg', cv2.resize(crop_img, (299, 299)))

        frame += 1
    cap.release()
    
    # 이비디오의 fake 영상만큼 반복
    for fake_vid in train_sample_metadata[train_sample_metadata['original']==vid].index:
        fake_path = train_sample + fake_vid
        cap2 = cv2.VideoCapture(fake_path)
        fake_vid_name = fake_vid.split('.')[0]
        frame2 = 0
        while(cap2.isOpened()):
            ret2 = cap2.grab()
            if ret2 == False:
                break
            #if frame2 % 30 == 0:
            ret2, image2 = cap2.retrieve()
            #img2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
            for face_idx, i in enumerate(face_coord[frame2]):
                y1,y2,x1,x2 = i
                crop_img3 = image2[y1:y2, x1:x2]
                cv2.imwrite(f'../working/dataset/FAKE3/FAKE_{fake_vid_name}_{frame2}_{face_idx}_{file_name}.jpg', cv2.resize(crop_img3, (299, 299)))
            frame2 += 1
        cap2.release()

        
    print(f'{vid_num}/{all_vid}')
    vid_num += 1
    

0/86


1/86


2/86


3/86


4/86


...

13/86


14/86


15/86


16/86


KeyboardInterrupt: 

In [16]:
y1

-2

In [17]:
y2

148

In [18]:
y11

-2

In [19]:
coord[2]

1297

In [20]:
coord[0]

7